In [1]:
# -*- coding: utf-8 -*-
"""
4_Model_Training_Colab_BART_v_Real_Metrics.py

Loads preprocessed data, loads BART-large, sets up Trainer.
Uses the REAL compute_metrics function with robust logging.
Designed for Google Colab with GPU acceleration.

Instructions:
1. !! IMPORTANT: Go to Runtime > Restart runtime... !!
2. Run ALL cells sequentially from Cell 1 downwards.
3. Verify Cell 11 output shows correct ROUGE metric loading status.
4. Verify Cell 13 output shows the correct compute_metrics function is passed.
5. Run Cell 15. Check evaluation logs for ROUGE scores and internal compute_metrics logs.
"""

# %% [code]
# =============================================================================
# Cell 1: Install Libraries
# =============================================================================
print("Handling gcsfs version...")
# Fix potential gcsfs incompatibility in Colab
!pip uninstall gcsfs -y -q
!pip install gcsfs==2024.6.1 -q
!pip show gcsfs # Verify version

print("\nInstalling other necessary libraries...")
# transformers[torch] installs transformers + pytorch
# accelerate is needed for fp16/distributed training optimizations
# rouge-score is required by the evaluate library
# nltk is needed for sentence tokenization for rouge
# sentencepiece might be needed by some tokenizers (e.g., T5's, included for safety)
!pip install transformers[torch] datasets evaluate rouge-score nltk accelerate sentencepiece -q
print("Libraries installed.")

# %% [code]
# =============================================================================
# Cell 2: Imports
# =============================================================================
import os
import sys
import datetime
import logging
import torch
import nltk
import numpy as np
import pandas as pd
import time
import traceback # For printing tracebacks in case of error

# Check if running in Colab for Drive mount
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Datasets/Transformers imports
from datasets import load_from_disk, DatasetDict
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate

print("Imports complete.")

# %% [code]
# =============================================================================
# Cell 3: Mount Google Drive (if in Colab)
# =============================================================================
DRIVE_BASE = None
if IN_COLAB:
    print("Attempting to mount Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully at /content/drive")
        DRIVE_BASE = '/content/drive/MyDrive/'
    except Exception as e:
        print(f"ERROR: Failed to mount Google Drive: {e}", file=sys.stderr)
        print("WARNING: Proceeding without Google Drive. Paths must be local Colab paths.")
else:
    print("Not running in Colab environment. Google Drive mount skipped.")

# %% [code]
# =============================================================================
# Cell 4: Configuration & Paths
# =============================================================================
print("--- Configuring Paths and Parameters ---")

# --- Paths (!! MODIFY PROJECT_FOLDER !!) ---
PROJECT_FOLDER = 'CAPSTONE 2 - Email Subject Line Generation/DATASET' # EXAMPLE

# --- Storage Option ---
USE_DRIVE_STORAGE = True # Set to False to force local /content/

if USE_DRIVE_STORAGE and DRIVE_BASE:
    BASE_PATH = os.path.join(DRIVE_BASE, PROJECT_FOLDER)
    print(f"Using Google Drive Path: {BASE_PATH}")
    STORAGE_TYPE = "Google Drive"
elif IN_COLAB:
    BASE_PATH = '/content/email_subject_project_data'
    print(f"Using Local Colab Storage Path: {BASE_PATH}")
    STORAGE_TYPE = "Local Colab Storage"
    os.makedirs(BASE_PATH, exist_ok=True)
    print(f"WARNING: DOWNLOAD results from {BASE_PATH} before runtime exit!")
else:
    BASE_PATH = './email_subject_project_data'
    print(f"Using Local Disk Storage Path: {BASE_PATH}")
    STORAGE_TYPE = "Local Disk Storage"
    os.makedirs(BASE_PATH, exist_ok=True)

if not os.path.isdir(BASE_PATH):
     print(f"ERROR: Base project folder not found: {BASE_PATH}", file=sys.stderr)
     raise SystemExit("Base path validation failed.")

PROCESSED_DATA_PATH = os.path.join(BASE_PATH, 'processed_datasets')
MODEL_OUTPUT_BASE_DIR = os.path.join(BASE_PATH, 'training_output')

# --- Model Configuration ---
MODEL_CHECKPOINT = "facebook/bart-large"
PREFIX = ""

# --- Preprocessing Lengths ---
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 32

run_description = "LearningRate_1e-5"

# --- Training Hyperparameters ---
NUM_EPOCHS = 3
TRAIN_BATCH_SIZE = 2
EVAL_BATCH_SIZE = 4
LEARNING_RATE =1e-5 # <--- Changed Learning Rate # original 2e-5
WEIGHT_DECAY = 0.01
GRADIENT_ACCUMULATION_STEPS = 4
SAVE_LIMIT = 1
EVAL_SAVE_LOG_STRATEGY = "epoch"
FP16_ENABLED = torch.cuda.is_available()

# --- Create unique output dir ---
run_timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
model_name_part = MODEL_CHECKPOINT.split('/')[-1]

#MODEL_OUTPUT_DIR = os.path.join(MODEL_OUTPUT_BASE_DIR, f"{model_name_part}#-finetune-{run_timestamp}")

MODEL_OUTPUT_DIR = os.path.join(MODEL_OUTPUT_BASE_DIR, f"{model_name_part}-finetune-{run_description}-{run_timestamp}")

# --- Logging Setup Variables ---
LOG_FILENAME = f"training_log_{run_timestamp}.txt"
LOG_FILE_PATH = os.path.join(MODEL_OUTPUT_DIR, LOG_FILENAME)

# --- Set up logging configuration ---
try:
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    log_handlers = [logging.FileHandler(LOG_FILE_PATH), logging.StreamHandler(sys.stdout)]
    print(f"Logging to console and file: {LOG_FILE_PATH}")
except OSError as e:
    print(f"ERROR: Could not create output directory {MODEL_OUTPUT_DIR}: {e}. Logging to console only.", file=sys.stderr)
    log_handlers = [logging.StreamHandler(sys.stdout)]

for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
# Use INFO level for normal runs, DEBUG for more detail (like inside compute_metrics)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=log_handlers)

#logging.info("--- Configuration Complete ---")
logging.info(f"--- Configuration Complete (Run Desc: {run_description}) ---")
print(f"--- Configuration Complete (Run Desc: {run_description}) ---")

logging.info(f"Storage Location: {STORAGE_TYPE} at {BASE_PATH}")
logging.info(f"Processed Data Path: {PROCESSED_DATA_PATH}")
logging.info(f"This Run Output Dir: {MODEL_OUTPUT_DIR}")
# ... (log other parameters as needed) ...
logging.info(f"Save Checkpoint Limit: {SAVE_LIMIT}")
logging.info("-" * 30)

# %% [code]
# =============================================================================
# Cell 5: Check GPU Availability
# =============================================================================
if not torch.cuda.is_available():
    logging.warning("GPU NOT FOUND! Training will be VERY SLOW.")
    FP16_ENABLED = False
else:
    logging.info(f"GPU Found: {torch.cuda.get_device_name(0)}")
    logging.info(f"Setting FP16 Enabled based on GPU availability: {FP16_ENABLED}")

# %% [code]
# =============================================================================
# Cell 6: NLTK Download for ROUGE
# =============================================================================
nltk_available = False
nltk_punkt_downloaded = False
try:
    import nltk
    nltk_available = True
    logging.info("NLTK library found.")
    try:
        logging.info("Downloading NLTK 'punkt' and 'punkt_tab' for ROUGE scoring...")
        nltk.download('punkt', quiet=True)
        nltk.download('punkt_tab', quiet=True, raise_on_error=False)
        logging.info("NLTK resources downloaded/verified.")
        nltk_punkt_downloaded = True
    except Exception as e:
        logging.error(f"Failed during NLTK download: {e}")
        logging.warning("ROUGE score calculation might be affected without NLTK data.")
except ImportError:
    logging.warning("NLTK library not installed. Cannot download resources for ROUGE.")

# %% [code]
# =============================================================================
# Cell 7: Load Processed Dataset
# =============================================================================
logging.info(f"Loading processed dataset from: {PROCESSED_DATA_PATH}")
tokenized_datasets = None
try:
    if not os.path.exists(PROCESSED_DATA_PATH): raise FileNotFoundError(f"Processed data dir not found: {PROCESSED_DATA_PATH}.")
    tokenized_datasets = load_from_disk(PROCESSED_DATA_PATH)
    logging.info("Processed dataset loaded successfully:"); logging.info(tokenized_datasets)
    if "train" not in tokenized_datasets or "dev" not in tokenized_datasets or "test" not in tokenized_datasets: raise ValueError("Missing train/dev/test split.")
    if len(tokenized_datasets['train']) == 0: raise ValueError("Training dataset empty.")
    required_cols = {'input_ids', 'attention_mask', 'labels'};
    if not required_cols.issubset(tokenized_datasets['train'].column_names): raise ValueError(f"Dataset missing required columns. Found: {tokenized_datasets['train'].column_names}")
except Exception as e: logging.error(f"Failed to load dataset: {e}", exc_info=True); raise SystemExit("Dataset loading failed.")

# %% [code]
# =============================================================================
# Cell 8: Load Tokenizer
# =============================================================================
logging.info(f"Loading tokenizer for checkpoint: {MODEL_CHECKPOINT}")
tokenizer = None
try:
    if tokenized_datasets is None: raise ValueError("Dataset not loaded before tokenizer.")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True)
    logging.info("Tokenizer loaded successfully.")
except Exception as e: logging.error(f"Failed to load tokenizer: {e}", exc_info=True); raise SystemExit("Tokenizer loading failed.")

# %% [code]
# =============================================================================
# Cell 9: Load Model
# =============================================================================
logging.info(f"Loading Sequence-to-Sequence model: {MODEL_CHECKPOINT}")
model = None
try:
    if tokenizer is None: raise ValueError("Tokenizer not loaded before model.")
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
    logging.info("Model loaded successfully.")
    logging.info(f"Model Parameter Count: {model.num_parameters()/1_000_000:.1f} M")
except Exception as e: logging.error(f"Failed to load model: {e}", exc_info=True); raise SystemExit("Model loading failed.")

# %% [code]
# =============================================================================
# Cell 10: Initialize Data Collator
# =============================================================================
logging.info("Initializing Data Collator for Seq2Seq...")
if model is None or tokenizer is None: raise SystemExit("Model or Tokenizer not loaded for Data Collator.")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
logging.info("Data Collator initialized.")

# %% [code]
# =============================================================================
# Cell 11: Setup Evaluation Metric (ROUGE) & Define compute_metrics (REAL VERSION)
# =============================================================================
# --- This cell contains the REAL compute_metrics function ---
logging.info("Loading ROUGE metric from 'evaluate' library...")
rouge_metric = None # Initialize
metric_load_error = None # Variable to store potential error
loaded_metric = None # Initialize loaded_metric outside try

try:
    # Attempt to load the metric
    loaded_metric = evaluate.load("rouge")
    logging.info(f"evaluate.load('rouge') call returned object of type: {type(loaded_metric)}")

    # --- CORRECTED CHECK: Only check if it's None ---
    if loaded_metric is not None:
        rouge_metric = loaded_metric # Assign the loaded object directly
        logging.info("Assigning loaded object to rouge_metric variable.")
    else:
        metric_load_error = "evaluate.load('rouge') explicitly returned None."
        logging.error(metric_load_error)
        # rouge_metric remains None
    # --- END CORRECTED CHECK ---

except Exception as e:
    metric_load_error = f"Exception during evaluate.load('rouge'): {e}"
    logging.error(metric_load_error, exc_info=True) # Log traceback
    # rouge_metric remains None

# --- Final check on the rouge_metric variable ---
if rouge_metric:
    logging.info("ROUGE metric object IS assigned and ready for use.")
    print("DEBUG: ROUGE metric object IS assigned.")
else:
    logging.error(f"ROUGE metric object IS NONE after checks. ROUGE scores might not be calculated. Last known reason: {metric_load_error}")
    print(f"DEBUG: ROUGE metric object IS NONE. Last known reason: {metric_load_error}")


# --- compute_metrics function definition (REAL VERSION with robust logging) ---
def compute_metrics(eval_pred):
    """Computes ROUGE scores for evaluation with detailed logging."""
    logging.info(f"--- compute_metrics called ---")
    logging.debug(f"Is rouge_metric None? {rouge_metric is None}")
    rouge_scores = {} # Default return

    if rouge_metric is None:
        logging.warning("compute_metrics: rouge_metric is None, returning empty scores {}.")
        return rouge_scores

    try:
        logging.debug("compute_metrics: rouge_metric is not None, entering main try block.")
        predictions, labels = eval_pred
        logging.debug("compute_metrics: Decoding predictions...")
        preds = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        logging.debug("compute_metrics: Decoding labels...")
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        logging.debug(f"compute_metrics: Decoded {len(decoded_preds)} pairs.")

        logging.debug(f"compute_metrics: Attempting NLTK tokenization (nltk_available={nltk_available}, nltk_punkt_downloaded={nltk_punkt_downloaded})...")
        if nltk_available and nltk_punkt_downloaded:
             try:
                decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
                decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
                logging.debug("compute_metrics: NLTK tokenization successful.")
             except Exception as nltk_e:
                 if not getattr(compute_metrics, "nltk_warning_logged", False):
                      logging.warning(f"NLTK sent_tokenize failed: {nltk_e}. Using fallback.")
                      compute_metrics.nltk_warning_logged = True
                 decoded_preds = [pred.strip() for pred in decoded_preds]
                 decoded_labels = [label.strip() for label in decoded_labels]
                 logging.debug("compute_metrics: NLTK tokenization failed, used fallback.")
        else:
            decoded_preds = [pred.strip() for pred in decoded_preds]
            decoded_labels = [label.strip() for label in decoded_labels]
            logging.debug("compute_metrics: NLTK unavailable/data missing, using raw decoded strings.")

        logging.debug(f"compute_metrics: Calling rouge_metric.compute() on {len(decoded_preds)} items...")
        if rouge_metric is None: # Double check before calling
             logging.error("compute_metrics: rouge_metric became None unexpectedly before compute call!")
             return {}

        result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        logging.debug(f"compute_metrics: rouge_metric.compute() returned: {result}")

        if result is None or not isinstance(result, dict) or not result:
             logging.error(f"compute_metrics: rouge_metric.compute() returned invalid result: {result}. Returning empty dict.")
             return {}

        logging.debug("compute_metrics: Processing compute() results...")
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.median(prediction_lens)
        rouge_scores = {key: round(value, 4) for key, value in result.items()}
        logging.info(f"compute_metrics: Final calculated scores: {rouge_scores}") # Use INFO for final scores

    except Exception as metric_e:
        logging.error(f"ERROR during main try block in compute_metrics: {metric_e}", exc_info=True)
        rouge_scores = {}

    logging.info(f"--- compute_metrics finished, returning: {rouge_scores} ---")
    return rouge_scores

logging.info("Compute_metrics function defined (REAL VERSION with robust logging).")


# %% [code]
# =============================================================================
# Cell 12: Define Training Arguments
# =============================================================================
logging.info("Defining Training Arguments...")
if 'MODEL_OUTPUT_DIR' not in locals() or 'NUM_EPOCHS' not in locals(): raise SystemExit("Required vars missing for TrainingArguments.")

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    eval_strategy=EVAL_SAVE_LOG_STRATEGY,
    save_strategy=EVAL_SAVE_LOG_STRATEGY,
    logging_strategy=EVAL_SAVE_LOG_STRATEGY,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH + 2,
    fp16=FP16_ENABLED,
    save_total_limit=SAVE_LIMIT,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL", # Use 'rougeL' as the key metric
    greater_is_better=True, # Higher ROUGE is better
    push_to_hub=False,
    report_to=["tensorboard"],
    logging_dir=os.path.join(MODEL_OUTPUT_DIR, 'logs'),
    seed=42,
)
logging.info("Training Arguments defined.")
logging.info(f"Effective Global Batch Size: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS * training_args.world_size}")
logging.info(f"Metric for best model: {training_args.metric_for_best_model}")

# %% [code]
# =============================================================================
# Cell 13: Initialize Trainer
# =============================================================================
logging.info("Initializing Trainer...")
compute_metrics_arg = None
if 'compute_metrics' in locals() and callable(compute_metrics):
     logging.info("DEBUG: compute_metrics function exists. Passing it to Trainer.")
     compute_metrics_arg = compute_metrics
else:
     logging.error("DEBUG: compute_metrics function NOT FOUND! Passing compute_metrics=None.")

if model is None or training_args is None or tokenized_datasets is None or tokenizer is None or data_collator is None:
     raise SystemExit("One or more required components for Trainer not ready.")

try:
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["dev"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics_arg, # Pass the real function
    )
    logging.info("Trainer initialized successfully.")
    logging.info(f"Using FP16 for training: {training_args.fp16}")
    logging.info(f"Trainer compute_metrics function set to: {trainer.compute_metrics}")
except Exception as e:
     logging.error(f"Failed to initialize Seq2SeqTrainer: {e}", exc_info=True)
     raise SystemExit("Trainer initialization failed.")

# %% [code]
# =============================================================================
# Cell 14: Ready for Training Info
# =============================================================================
logging.info("Setup complete. Ready to start training in the next cell.")
print("\n" + "="*50)
print(" NEXT STEP: Execute the next cell to run trainer.train()")
print("           (Using REAL compute_metrics function)")
print("="*50)

# %% [code]
# =============================================================================
# Cell 15: Execute Training, Save Model, Evaluate (REAL METRICS)
# =============================================================================
logging.info("="*50)
logging.info("          STARTING MODEL TRAINING & EVALUATION (REAL METRICS)          ")
logging.info("="*50)
print("\n--- Starting Training (REAL Metrics) ---")

train_result = None
training_duration_seconds = 0
start_time = time.time()
start_timestamp_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
logging.info(f"Starting trainer.train() at: {start_timestamp_str}")

try:
    if 'trainer' not in locals() or trainer is None: raise RuntimeError("Trainer object not initialized.")
    train_result = trainer.train()
    print("\n--- Training finished (REAL Metrics) ---")
    logging.info("Training execution completed.")
except Exception as training_e:
    logging.exception(f"An error occurred during trainer.train(): {training_e}")
    print(f"ERROR during training: {training_e}", file=sys.stderr)
finally:
    end_time = time.time()
    end_timestamp_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    training_duration_seconds = end_time - start_time
    training_duration_formatted = str(datetime.timedelta(seconds=training_duration_seconds))
    logging.info(f"Training block finished at: {end_timestamp_str}")
    logging.info(f"Total training block duration: {training_duration_formatted} (Sec: {training_duration_seconds:.2f})")

if train_result:
    try:
        logging.info("Saving best model (based on real metrics) and trainer state...")
        print("Saving best model and trainer state...")
        trainer.save_model()
        trainer.save_state()
        logging.info(f"Model and state saved to: {MODEL_OUTPUT_DIR}")
        print(f"Model and state saved to: {MODEL_OUTPUT_DIR}")

        final_train_metrics = train_result.metrics
        final_train_metrics["train_duration_seconds"] = round(training_duration_seconds, 2)
        final_train_metrics["train_duration_formatted"] = training_duration_formatted
        logging.info("Logging final training run metrics...")
        trainer.log_metrics("train", final_train_metrics)
        try:
            trainer.save_metrics("train", final_train_metrics)
            logging.info(f"Training metrics saved to {os.path.join(MODEL_OUTPUT_DIR, 'train_results.json')}")
        except Exception as e: logging.error(f"Failed to save training metrics: {e}")

        logging.info("\n--- Evaluating final model on TEST set (Using REAL Metrics) ---")
        print("\n--- Evaluating final model on TEST set (Using REAL Metrics) ---")
        if "test" in tokenized_datasets:
            try:
                test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"], metric_key_prefix="test")
                logging.info("Logging test metrics...")
                trainer.log_metrics("test", test_metrics)
                try:
                    trainer.save_metrics("test", test_metrics)
                    logging.info(f"Test metrics saved to {os.path.join(MODEL_OUTPUT_DIR, 'test_results.json')}")
                except Exception as e: logging.error(f"Failed to save test metrics: {e}")

                log_and_print = lambda msg: (logging.info(msg), print(msg))
                log_and_print("\n" + "="*30); log_and_print("  Final Test Set Evaluation Results (REAL METRICS)"); log_and_print("="*30)
                for key, value in test_metrics.items(): log_line = f"  {key}: {value}"; log_and_print(log_line)
                log_and_print("="*30)
            except Exception as eval_e: logging.error(f"Error during final test evaluation: {eval_e}", exc_info=True)
        else: logging.warning("Test dataset not found. Skipping final test evaluation.")

        logging.info(f"\n{'='*15} Script Finished Successfully (Real Metrics) {'='*15}")
        print(f"\n{'='*15} Script Finished Successfully (Real Metrics) {'='*15}")
    except Exception as post_train_e:
        logging.exception(f"Error during post-training: {post_train_e}")
        print(f"ERROR during post-training: {post_train_e}", file=sys.stderr)
else:
    logging.error("Training did not complete successfully. Check logs.")
    print("ERROR: Training did not complete successfully.", file=sys.stderr)

# %% [code]
# =============================================================================
# Cell 16: Cleanup Logger (Optional)
# =============================================================================
# try:
#     if 'logger' in locals() and isinstance(logging.getLogger(), logging.RootLogger) and logging.getLogger().hasHandlers():
#          print("Flushing and closing log handlers...")
#          logging.shutdown()
#          print("Logger handlers flushed and closed.")
# except Exception as e: print(f"Error during logger cleanup: {e}", file=sys.stderr)

Handling gcsfs version...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


2025-04-20 21:03:24,339 - INFO - --- compute_metrics called ---
2025-04-20 21:03:24,422 - INFO - Using default tokenizer.
2025-04-20 21:03:25,389 - INFO - compute_metrics: Final calculated scores: {'rouge1': np.float64(0.3668), 'rouge2': np.float64(0.1884), 'rougeL': np.float64(0.3572), 'rougeLsum': np.float64(0.3574), 'gen_len': np.float64(11.0)}
2025-04-20 21:03:25,390 - INFO - --- compute_metrics finished, returning: {'rouge1': np.float64(0.3668), 'rouge2': np.float64(0.1884), 'rougeL': np.float64(0.3572), 'rougeLsum': np.float64(0.3574), 'gen_len': np.float64(11.0)} ---
2025-04-20 21:27:04,427 - INFO - --- compute_metrics called ---
2025-04-20 21:27:04,524 - INFO - Using default tokenizer.
2025-04-20 21:27:05,493 - INFO - compute_metrics: Final calculated scores: {'rouge1': np.float64(0.3648), 'rouge2': np.float64(0.1887), 'rougeL': np.float64(0.3565), 'rougeLsum': np.float64(0.3564), 'gen_len': np.float64(11.0)}
2025-04-20 21:27:05,494 - INFO - --- compute_metrics finished, return

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



--- Training finished (REAL Metrics) ---
2025-04-20 21:27:48,788 - INFO - Training execution completed.
2025-04-20 21:27:48,789 - INFO - Training block finished at: 2025-04-20 21:27:48
2025-04-20 21:27:48,790 - INFO - Total training block duration: 1:11:10.364548 (Sec: 4270.36)
2025-04-20 21:27:48,792 - INFO - Saving best model (based on real metrics) and trainer state...
Saving best model and trainer state...
2025-04-20 21:28:03,032 - INFO - Model and state saved to: /content/drive/MyDrive/CAPSTONE 2 - Email Subject Line Generation/DATASET/training_output/bart-large-finetune-LearningRate_1e-5-20250420_201611
Model and state saved to: /content/drive/MyDrive/CAPSTONE 2 - Email Subject Line Generation/DATASET/training_output/bart-large-finetune-LearningRate_1e-5-20250420_201611
2025-04-20 21:28:03,034 - INFO - Logging final training run metrics...
***** train metrics *****
  epoch                    =         2.9986
  total_flos               =     16808504GF
  train_duration_formatted 

2025-04-20 21:31:18,676 - INFO - --- compute_metrics called ---
2025-04-20 21:31:18,780 - INFO - Using default tokenizer.
2025-04-20 21:31:19,791 - INFO - compute_metrics: Final calculated scores: {'rouge1': np.float64(0.355), 'rouge2': np.float64(0.1868), 'rougeL': np.float64(0.3447), 'rougeLsum': np.float64(0.3451), 'gen_len': np.float64(16.0)}
2025-04-20 21:31:19,792 - INFO - --- compute_metrics finished, returning: {'rouge1': np.float64(0.355), 'rouge2': np.float64(0.1868), 'rougeL': np.float64(0.3447), 'rougeLsum': np.float64(0.3451), 'gen_len': np.float64(16.0)} ---
2025-04-20 21:31:19,806 - INFO - Logging test metrics...
***** test metrics *****
  epoch                   =     2.9986
  test_gen_len            =       16.0
  test_loss               =     2.4927
  test_rouge1             =      0.355
  test_rouge2             =     0.1868
  test_rougeL             =     0.3447
  test_rougeLsum          =     0.3451
  test_runtime            = 0:03:16.73
  test_samples_per_second =